# Part 4: ETL dan Scheduled Job

## Import and Load

In [1]:
%load_ext google.cloud.bigquery

In [2]:
import os
import pandas as pd
from google.cloud import bigquery
import google.auth
from google.cloud import bigquery

In [3]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'C:/Users\Asus/Documents/DE-CodingID/aqueous-freedom-398208-cfe0d4e567f2.json'

In [4]:
bigquery_client = bigquery.Client()

## 1. Segmentasi customer berdasarkan jumlah order dan uang yang sudah dikeluarkan di platform OK! Food Delivery? Ini akan menjadi dasar untuk tim marketing dalam upaya menjaga mereka agar mau terus menggunakan platform OK! Food Delivery.

In [23]:
query1 = """    
    WITH kategori AS (
        SELECT 
            c.name, 
            COUNT(*) AS total_order, 
            SUM(f.price*o.quantity) AS total_spend
        FROM `aqueous-freedom-398208.OFD_Team_1.Order` o
        INNER JOIN `aqueous-freedom-398208.OFD_Team_1.Food` f ON f.id_food = o.fk_food_id
        INNER JOIN `aqueous-freedom-398208.OFD_Team_1.Customer` c ON c.id_customer = o.fk_customer_id
        GROUP BY 1
    )
    
    SELECT 
        k.name, 
        k.total_order, 
        k.total_spend, 
        CASE
            WHEN k.total_order > 3 AND k.total_spend >= 150000 THEN 'Big Spender'
            WHEN k.total_spend < 50000 THEN 'Regular'
            WHEN k.total_spend < 100000 THEN 'Silver'
            WHEN k.total_spend < 150000 THEN 'VIP'
            ELSE 'VVIP' END level_segmentasi
    FROM kategori AS k
    ORDER BY 2, 3, 4;
"""

result1 = bigquery_client.query(query1)
df1 = result1.to_dataframe()
df1

,name,total_order,total_spend,level_segmentasi
0,Ridho,1,30000,Regular
1,David,1,105000,VIP
2,Fauzan,2,110000,VIP
3,Heru,2,120000,VIP
4,Jamal,2,122000,VIP
5,Fine,2,125000,VIP
6,Fadhli,2,140000,VIP
7,Fahmi,2,165000,VVIP
8,Danu,3,66000,Silver
9,Badu,3,79000,Silver


## 2. Semua transaksi yang masih ‘menggantung’ dari Day-1 agar bisa di follow up oleh tim operational / customer service

In [20]:
query2 = """
    SELECT 
        c.name, 
        o.order_date, 
        o.status,
        c.contact
    FROM `aqueous-freedom-398208.OFD_Team_1.Customer` c
    INNER JOIN `aqueous-freedom-398208.OFD_Team_1.Order` o ON c.id_customer = o.fk_customer_id
    WHERE o.status = 'pending'
    ORDER BY o.order_date ASC;
"""

result2 = bigquery_client.query(query2)
df2 = result2.to_dataframe()
df2

,name,order_date,status,contact
0,Budi,2023-11-04,pending,3091
1,Harun,2023-11-06,pending,6942
2,Badu,2023-11-07,pending,5726
3,Inul,2023-11-11,pending,5493
4,Eric,2023-11-12,pending,2947
5,Harun,2023-11-13,pending,6942
6,Dika,2023-11-14,pending,1239
7,Haris,2023-11-16,pending,4632
8,Geri,2023-11-19,pending,3816
9,Eti,2023-11-20,pending,8304
